In [9]:
import sys
import os

# 把 TaggerCore 目錄添加到 sys.path
sys.path.append(os.path.join(os.getcwd(), 'TaggerCore'))

In [10]:
sys.path

['/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/home/schoenhsiung/.virtualenvs/py38/lib/python3.8/site-packages',
 '/home/schoenhsiung/crawler/Z_download/product_tagger/TaggerCore']

In [22]:
import json
import pandas as pd
from data.test_data import grp_data, etk_data
from TaggerCore.tagger import ProductTagger
import requests
import ast

In [12]:
# demo partial data from .xlxs file
title = grp_data['title']
title

['首爾旅遊｜賞楓內藏山.雞籠山國立公園.華蓋庭園.江華島傳燈寺.青瓦臺.全州韓屋村.明洞(二站購物)六日',
 '洄來花蓮玩｜五星西班牙美學理想大地、歐風水岸星巴克、飄飄雲朵、羅山瀑布、小吃美食二日｜台南高雄出發',
 '追楓東北|超人氣景點~只見線橋梁美景.藏王纜車.五色沼遊湖.塔之崖.秋保瀑布.會津若松城.仙台牛舌餐.溫泉五日',
 '魅力歐洲｜德瑞法｜海德古堡纜車、萊茵河三遊船、凱旋門登頂趣、全覽鐵力士山10日',
 '花蓮旅遊｜11/02、11/03限定｜洄瀾漾起來.東大門【一路向東】演唱會.出遊挺花蓮二日｜台中出發',
 '東京清倉特惠｜富士山美景.忍野八海.麻布台之丘.豐洲千客萬來五日']

In [13]:
# Load tag definition
with open('./data/activity_tag.json') as f:
    data = json.load(f)

In [14]:
def create_and_tag_product(model_name, model_api_base, format_args):
    """
    Create a ProductTagger, format the messages, and tag the product synchronously.
    """
    tagger = ProductTagger(model_name, model_api_base)
    tagger.format_messages(format_args)
    result = tagger.Tag()  # Synchronous call to Tag()
    return result

In [ ]:
## def create_and_tag_product() 步驟拆解
from TaggerCore import prompts
from transformers import AutoTokenizer

# tagger basic saetting
SYS_PROMPT = prompts.SYS_PROMPT
USER_PROMPT = prompts.USER_PROMPT

model_name = "Qwen/Qwen2.5-32B-Instruct"
print('Loading Tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(model_name)
print('Tokenizer Loaded.')
model_api_base = "http://10.34.6.6:8000/generate"

messages = []

# format massage
format_args_2 = {
    'tag_definition': data['放鬆'],
    'product_info': title[4]
}
messages = [
    {"role": "system", "content": SYS_PROMPT},
    {"role": "user", "content": USER_PROMPT.format(**format_args_2)}
]
__cache_messages = [
    {"role": "system", "content": SYS_PROMPT},
    {"role": "user", "content": USER_PROMPT.format(**format_args_2)}
]
print('Messages Formatted.')

# tag
prompt = tokenizer.apply_chat_template(    # tokenizer.apply_chat_template(): 主要用來格式化對話（Chat）類型的輸入，使其適合用於模型推理（Inference）。
    messages,
    tokenize=False,    # False: 返回格式化後的純文字字串（適合人工檢查）、True: 返回的是 Token ID（適合模型推理）
    add_generation_prompt=True    # 是否自動添加生成提示。True: 會自動在對話結尾加上模型應該繼續生成的提示，適合用於聊天機器人的輸入，確保模型知道它應該產生下一個回應。
)

# output = self.chatbot.chat(prompt)  # Asynchronous chat
params = {
    "max_new_tokens": 2000,  # 生成的最大 Token 數量
    "temperature": 0.5,  # 溫度，決定隨機性
    "top_p": 0.5,  # 核採樣
    "top_k": 85,  # 取前 K 個最高機率的 Token
    "repetition_penalty": 1.0,  # 重複懲罰，防止模型重複內容
    "do_sample": True,  # 啟用隨機取樣
}
# model_api_base = <your api url>
resp = requests.post(model_api_base, json=params)  # request model
text = resp.json()["text"][0]  # get return text
# parse output response
response = text[text.rfind('<|im_start|>assistant\n') + 22:]
response_json = ast.literal_eval(response)

Loading Tokenizer...
Tokenizer Loaded.
Messages Formatted.
